# Mario Filho Live \#24: Intro PyTorch

Porque usar PyTorch:
+ Tem boas ferramentas OpenSource
+ Tem uma boa documentação

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

A unidade básica do PyTorch é um tensor: É como um numpy.array

In [ ]:
# Criar um Tensor
t = torch.Tensor([1,2,3])
t

tensor([1., 2., 3.])

In [ ]:
a = np.array([1,2,3])
t = torch.from_numpy(a)
t

tensor([1, 2, 3])

In [ ]:
# Multiplicar um tensor é o mesmo que multiplicar uma array
#   ou seja, todaos os números
t * 10

tensor([10, 20, 30])

In [ ]:
t ** 2

tensor([1, 4, 9])

In [ ]:
print(t)
t.dot(t)

tensor([1, 2, 3])


tensor(14)

## Testando com dataSets

DataSet para fazer regressão  de preços de casas.

In [ ]:
data = pd.read_csv("sample_data/california_housing_train.csv")
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [ ]:
# Tirando o 'y' final
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

## Como montar a rede neural

In [ ]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    # Definindo a estrutura da nossa rede
    # 8 inputs -> 100 -> 100 -> 1
    
    # 8 entradas ligadas a 100 neuronios: todos terao conexões entre todos
    self.fc1 = nn.Linear(8, 100) 

    # Ligando todos os 100 neuronios em 100 neuronios próximos
    self.fc2 = nn.Linear(100, 100)

    # Camada de saida
    self.out = nn.Linear(100, 1)

  # processo para a rede processar um dado
  def forward(self, x):
    x = F.normalize(x) # Normalizar para melhorar a eficiência da rede. Usando regra L2
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    y = self.out(x)
    return y

In [ ]:
# Criando a Rede Neural
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=8, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
# Convertendo de numpy para Torch
X_tensor = torch.from_numpy(data.values[:, :-1]).float()
y_tensor = torch.from_numpy(data.values[:, -1]).float()

# Vamos aplicar o log sobre 'y' pois sem isso vai dar roblema
# pois os parametros estao normalizados mas o 'y' é muito alto
y_tensor = torch.log(y_tensor)

In [ ]:
# Otimizador: O responsável por treinar a rede e fazer os cálculos
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# O critério para determinar o erro: MSELoss = Erro médio
criterion = nn.MSELoss() # isso age como um decorator, nao da pra usar direto no treinamento

In [ ]:
# Processo de treinamento da rede, onde i = epoch
for i in range(100):
  # zerar os gradeintes, para que os gradientes da previsao anterior nao atrapaçhe
  optimizer.zero_grad()
  # Prever (fazer o foward) uma entrada
  prediction = net(X_tensor)
  # calcular o erro
  loss = criterion(prediction, y_tensor)
  # Calculo do gradiente
  loss.backward()
  # atualiza os pesos
  optimizer.step()
  
  # Vamos printar o erro,a loss a cada passos

  # exp para sair do 'log' do 'y'
  prediction2 = torch.exp(net(X_tensor))
  loss = criterion(prediction2, torch.exp(y_tensor))
  print("Monitor de loss:", torch.sqrt(loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([17000])) that is different to the input size (torch.Size([17000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6250, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.6094, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(

## Testando

In [ ]:
data_test = pd.read_csv("sample_data/california_housing_test.csv")
data_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [ ]:
X_test_tensor = torch.from_numpy(data_test.values[:, :-1]).float()
y_test = data_test.values[:, -1]


In [ ]:
# Prever os valores de teste
prediction_test = torch.exp(net(X_test_tensor))

In [ ]:
print(prediction_test)

tensor([[ 194207.6875],
        [ 289592.7812],
        [ 177786.3594],
        ...,
        [ 323395.7812],
        [1371242.0000],
        [ 223691.4062]], grad_fn=<ExpBackward>)


In [ ]:
prediction_test_np = prediction_test.detach().numpy()
prediction_test_np

array([[ 194207.69],
       [ 289592.78],
       [ 177786.36],
       ...,
       [ 323395.78],
       [1371242.  ],
       [ 223691.4 ]], dtype=float32)

In [ ]:
from sklearn.metrics import mean_squared_error
y_test = data_test.values[:, -1]
# Valor do erro médio quadrado
np.sqrt(mean_squared_error(y_test, prediction_test_np))
# Mario conseguiu resudir para
## 76790.5119855592

215448.27306136442